<a href="https://colab.research.google.com/github/Abhishek672/cs6910_Assignment2/blob/manideep2/DLCNN_2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from PIL import Image
import numpy as np
import pickle

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.notebook import tqdm

from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model

from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception


from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,AveragePooling2D,Dropout,BatchNormalization,Activation
from keras.activations import relu,softmax
from keras.initializers import he_normal,glorot_normal,random_normal,glorot_uniform
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta,Adam
from keras.utils import np_utils

import warnings
warnings.filterwarnings('ignore')


In [19]:
!nvidia-smi

Thu Apr  8 12:49:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    28W /  70W |   9052MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.3MB 42kB/s 


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
class Fine_Tune:

  #target_size = (w x h x channel)
  def __init__(self,target_size, num_neurons,num_hlayers,pretrained_model,trainable_layers):
    self.model = Sequential()
    self.target_size = target_size
    self.num_neurons = num_neurons
    self.num_hlayers = num_hlayers
    self.pretrained_model = pretrained_model
    self.trainable_layers = trainable_layers

  def BUILD_MODEL(self):
    if (self.pretrained_model == 'InceptionV3'):
      prt = InceptionV3(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 17
      
    elif (self.pretrained_model == 'InceptionResnetV2'):
      prt = InceptionResNetV2(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 17

    elif (self.pretrained_model == 'ResNet50'):
      prt = ResNet50(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 10

    elif (self.pretrained_model == 'Xception') :
      prt = Xception(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 6
    else:
      raise Exception("this model not used!!")
   
    if self.trainable_layers == 'No':
      for layer in prt.layers:
          layer.trainable = False
    else:
      for layer in prt.layers[:-1*trl]:
          layer.trainable = False

    self.model.add(prt)

    #Add dense layers.. 
    self.model.add(Flatten())

    for i in range(self.num_hlayers):
      self.model.add(Dense(units = self.num_neurons))
      self.model.add(Activation('relu'))

    self.model.add(Dense(units=10))
    self.model.add(Activation('softmax'))

    self.model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

    return


  def Fit_Model(self,train_path,epochs,batch_size):
    tr_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    horizontal_flip=True,
                                    validation_split = 0.2)

    tr_gen = tr_datagen.flow_from_directory(
            directory = train_path,target_size = self.target_size[:2],color_mode = 'rgb',       
            batch_size = batch_size,
            class_mode = 'categorical',
            shuffle = True,
            seed = 10,
            subset = 'training'
            )

    val_gen = tr_datagen.flow_from_directory(
            directory = train_path,target_size = self.target_size[:2],color_mode = 'rgb',
            batch_size = batch_size,
            class_mode = 'categorical',
            shuffle = True,
            seed = 10,
            subset = 'validation'
            ) 
   
    step_size_train = tr_gen.n//tr_gen.batch_size
    step_size_valid = val_gen.n//val_gen.batch_size

    self.model.fit_generator(generator = tr_gen,
                             steps_per_epoch = step_size_train,
                             validation_data = val_gen,
                             validation_steps = step_size_valid,
                             epochs = epochs,
                             #callbacks=[WandbCallback(monitor='val_accuracy',save_model=True)],
                             verbose = 1)
    return

  def Predict_Test(self,test_path):

    ts_datagen = ImageDataGenerator(rescale = 1./255)
    test_gen = ts_datagen.flow_from_directory(
        directory = test_path,
        target_size = self.target_size[:2],
        color_mode = 'rgb',
        batch_size = 1,
        class_mode = None,
        shuffle = False,
        seed=10
        )
    
    step_size_test = test_gen.n//test_gen.batch_size
    test_gen.reset()
    print('Getting predictions for test data..')
    pred = self.model.predict_generator(generator = test_gen,
                                  steps = step_size_test,
                                  verbose = 1
                                  )
    ytrue = test_gen.classes ; ypred = np.argmax(pred,axis=1)
    return np.sum(ytrue == ypred)/len(ytrue)

In [9]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb: Currently logged in as: manideepladi (use `wandb login --relogin` to force relogin)


True

**Question 2 (5 Marks)
You will notice that InceptionV3, InceptionResNetV2, ResNet50, Xception are very huge models as compared to the simple model that you implemented in Part A. Even fine-tuning on a small training data may be very expensive. What is a common trick used to keep the training tractable (you will have to read up a bit on this)? Try different variants of this trick and fine-tune the model using the iNaturalist dataset. For example, '___'ing all layers except the last layer, '___'ing upto k layers and '___'ing the rest. Read up on pre-training and fine-tuning to understand what exactly these terms mean.**

**Write down the different strategies that you tried (simple bullet points would be fine).**

In [16]:
sweep_config = {
  'name': 'Sweep for Assignment PartB',
  'method': 'grid',
  'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
  'parameters': {
        'num_neurons': {
            'values': [64,128]
        },
        'num_hlayers':{
            'values':[1,2]
        },
        'pretrained_model':{
            'values':['InceptionV3','Xception','InceptionResnetV2','ResNet50']
        },
        'trainable_layers':{
            'values':['Yes','No']
        }

    }
}

sweep_id = wandb.sweep(sweep_config, project='CNN_PartB', entity='manideepladi')

Create sweep with ID: u8v89osp
Sweep URL: https://wandb.ai/manideepladi/CNN_PartB/sweeps/u8v89osp


In [17]:
import wandb
from wandb.keras import WandbCallback

In [18]:
def train():
  run = wandb.init()
  config = run.config

  train_path = '/content/gdrive/MyDrive/inaturalist_12K_subset/train'

  target_size = (256,256,3)

  obj1 = Fine_Tune(target_size=target_size ,
                   pretrained_model = config.pretrained_model,
                   num_neurons = config.num_neurons,
                   num_hlayers = config.num_hlayers,
                   trainable_layers=config.trainable_layers) 
                       
  obj1.BUILD_MODEL()
  obj1.Fit_Model(train_path,epochs=10,batch_size=32)

wandb.agent(sweep_id=sweep_id, function=train )


wandb: Agent Starting Run: anebgb96 with config:
wandb: 	num_hlayers: 1
wandb: 	num_neurons: 64
wandb: 	pretrained_model: InceptionV3
wandb: 	trainable_layers: Yes


Found 1600 images belonging to 10 classes.
Found 400 images belonging to 10 classes.
Epoch 1/10
50/50 [==============================] - 712s 14s/step - loss: 4.4352 - accuracy: 0.3109 - val_loss: 1.5942 - val_accuracy: 0.5234
Epoch 2/10
50/50 [==============================] - 53s 1s/step - loss: 0.9674 - accuracy: 0.6904 - val_loss: 1.2215 - val_accuracy: 0.6667
Epoch 3/10
50/50 [==============================] - 52s 1s/step - loss: 0.6280 - accuracy: 0.8043 - val_loss: 1.2594 - val_accuracy: 0.6953
Epoch 4/10
50/50 [==============================] - 52s 1s/step - loss: 0.4147 - accuracy: 0.8654 - val_loss: 1.4873 - val_accuracy: 0.6536
Epoch 5/10
50/50 [==============================] - 52s 1s/step - loss: 0.3636 - accuracy: 0.8643 - val_loss: 1.4369 - val_accuracy: 0.7135
Epoch 6/10
50/50 [==============================] - 52s 1s/step - loss: 0.1974 - accuracy: 0.9335 - val_loss: 1.4520 - val_accuracy: 0.6979
Epoch 7/10
50/50 [==============================] - 52s 1s/step - loss: 0

epoch,9
loss,0.09884
accuracy,0.97062
val_loss,1.8823
val_accuracy,0.72135
_runtime,1200
_timestamp,1617868488
_step,9
best_val_accuracy,0.73438
best_epoch,8


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▁▁▁▁▁
accuracy,▁▅▆▇▇▇████
val_loss,▅▁▁▄▃▃▄▇▇█
val_accuracy,▁▆▇▅▇▇█▇██
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: 46egltko with config:
wandb: 	num_hlayers: 1
wandb: 	num_neurons: 64
wandb: 	pretrained_model: InceptionV3
wandb: 	trainable_layers: No


...

wandb: Agent Starting Run: difqpcke with config:
wandb: 	num_hlayers: 2
wandb: 	num_neurons: 128
wandb: 	pretrained_model: ResNet50
wandb: 	trainable_layers: No


Found 1600 images belonging to 10 classes.
Found 400 images belonging to 10 classes.
Epoch 1/10
50/50 [==============================] - 60s 1s/step - loss: 8.5700 - accuracy: 0.1068 - val_loss: 2.3958 - val_accuracy: 0.1068
Epoch 2/10
50/50 [==============================] - 54s 1s/step - loss: 2.6289 - accuracy: 0.1352 - val_loss: 3.0508 - val_accuracy: 0.1719
Epoch 3/10
50/50 [==============================] - 53s 1s/step - loss: 2.5867 - accuracy: 0.1535 - val_loss: 2.7494 - val_accuracy: 0.1484
Epoch 4/10
50/50 [==============================] - 54s 1s/step - loss: 2.4078 - accuracy: 0.1520 - val_loss: 2.3026 - val_accuracy: 0.1536
Epoch 5/10
50/50 [==============================] - 55s 1s/step - loss: 2.2285 - accuracy: 0.1845 - val_loss: 2.3699 - val_accuracy: 0.1693
Epoch 6/10
50/50 [==============================] - 53s 1s/step - loss: 2.2038 - accuracy: 0.1970 - val_loss: 2.1977 - val_accuracy: 0.2135
Epoch 7/10
50/50 [==============================] - 54s 1s/step - loss: 2.2

epoch,9
loss,2.17732
accuracy,0.2075
val_loss,2.35266
val_accuracy,0.16667
_runtime,553
_timestamp,1617886161
_step,9
best_val_accuracy,0.21354
best_epoch,5


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▁▁▁▁▁▁▁
accuracy,▁▂▃▅▇█▇▅██
val_loss,▃█▆▂▂▁▃▂▂▂
val_accuracy,▁▅▄▄▅█▃▄▆▅
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [8]:
train_path = '/content/gdrive/MyDrive/inaturalist_12K_subset/train'

target_size = (224,224,3)

obj1 = Fine_Tune(target_size=target_size ,
                   pretrained_model = "ResNet50",
                   num_neurons =64,
                   num_hlayers = 1,
                   trainable_layers="No") 
obj1.BUILD_MODEL()
obj1.Fit_Model(train_path,epochs=10,batch_size=32)

Found 1600 images belonging to 10 classes.
Found 400 images belonging to 10 classes.
Epoch 1/10
50/50 [==============================] - 975s 19s/step - loss: 6.1173 - accuracy: 0.1169 - val_loss: 2.3026 - val_accuracy: 0.1042
Epoch 2/10
50/50 [==============================] - 51s 1s/step - loss: 2.3023 - accuracy: 0.1106 - val_loss: 2.3025 - val_accuracy: 0.1042
Epoch 3/10
50/50 [==============================] - 51s 1s/step - loss: 2.3026 - accuracy: 0.1006 - val_loss: 2.3026 - val_accuracy: 0.0990
Epoch 4/10
50/50 [==============================] - 51s 1s/step - loss: 2.3027 - accuracy: 0.0993 - val_loss: 2.3027 - val_accuracy: 0.0938
Epoch 5/10
50/50 [==============================] - 51s 1s/step - loss: 2.3026 - accuracy: 0.1031 - val_loss: 2.3025 - val_accuracy: 0.1042
Epoch 6/10
50/50 [==============================] - 51s 1s/step - loss: 2.3026 - accuracy: 0.0984 - val_loss: 2.3026 - val_accuracy: 0.1016
Epoch 7/10
50/50 [==============================] - 51s 1s/step - loss: 2

In [ ]:
 #test_path = '/content/gdrive/MyDrive/inaturalist_12K_subset/val'
  #test_auc = obj1.Predict_Test(test_path)
  #wandb.log({"Test accuracy": test_auc})

**Question 3 (15 Marks)
Now finetune the model using different strategies that you discussed above and different hyperparameter choices. Based on these experiments write down some insightful inferences (once again you will find the sweep function to be useful to plot and compare different choices).**